In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import numpy as np
from statistics import mode
from fuzzywuzzy import fuzz

In [4]:
# import excel file and separate df by car make
xls = pd.ExcelFile('../Resources/Car_Data.xlsx')
df1 = pd.read_excel(xls, 'Volkswagon')
# df2 = pd.read_excel(xls, 'Toyota')
# df3 = pd.read_excel(xls, 'Porsche')
# df4 = pd.read_excel(xls, 'Mercedes')
# df5 = pd.read_excel(xls, 'Maserati')
# df6 = pd.read_excel(xls, 'Lincoln')
# df7 = pd.read_excel(xls, 'Lexus')
# df8 = pd.read_excel(xls, 'Kia')
# df9 = pd.read_excel(xls, 'Landrover')
# df10 = pd.read_excel(xls, 'Volkswagon')
# df11 = pd.read_excel(xls, 'Honda')
# df12 = pd.read_excel(xls, 'Hyundai')
# df13 = pd.read_excel(xls, 'Acura')
# df14 = pd.read_excel(xls, 'Audi')
# df15 = pd.read_excel(xls, 'BMW')

In [5]:
def clean_data(df):
    # drop null values
    new_df = df.dropna(how='any')
    # drop duplicates
    new_df = new_df.drop_duplicates()
    # reformat datetime to date_time object
    new_df['formatted_sold_date'] = pd.to_datetime(new_df['sold_date'], format='%Y-%m-%d-%H.%M.%S.%f')
    new_df['sold_date_year'] = new_df['formatted_sold_date'].dt.year
    new_df['sold_date_month'] = new_df['formatted_sold_date'].dt.month
    # drop unneeded columns (dc = drop columns)
    dc_df = new_df.drop(columns=['sold_date', 'formatted_sold_date', 'subseries', 'make', 'transmission', 'seller'])

    # replace condition grade text values with numbers
    dc_df['condition_grade'] = dc_df['condition_grade'].replace(
    {'SL': 0, 'RG': 20, 'PR': 10, 'EC':50, 'CL':40, 'C': 40, 'AV':30, 'A': 30, 'I0': 0})

    return dc_df


In [6]:
# df_list = {df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15}
#
# for df in df_list:
#     clean_[df] = clean_data(df)

clean_df1 = clean_data(df1)

In [7]:
# Generate our categorical variable lists
car_cat = clean_df1.dtypes[clean_df1.dtypes == "object"].index.tolist()

In [8]:
model_cnt = clean_df1[car_cat]["model"].value_counts()

In [9]:
# Group cars models less than 100 into 'other' category\n",
# Choose a cutoff value and create a list of application types to be replaced\n",
# use the variable name `application_types_to_replace`\n",
model_types_to_replace = list(model_cnt[model_cnt < 100].index)
# Replace in dataframe
for model in model_types_to_replace:
    clean_df1['model'] = clean_df1['model'].replace(model,"Other")
    clean_df1['model'] = clean_df1['model'].apply(str)


In [10]:
# for each unique value in Description
for d in clean_df1.model.unique():
    # compute Levensthein distance
    # and set to True if >= 70
    # (you may have to play around with it)
    clean_df1[d] = clean_df1['model'].apply(
        lambda x : fuzz.ratio(x, d) >= 70
    )
        # set a name for the group\n",
        # here, simply the shortest\n",
    m = np.min(clean_df1[clean_df1[d]==True].model)
    # assign the group\n",
    clean_df1.loc[clean_df1.model==d, 'model_group'] = m

In [11]:
color_cnt = clean_df1['color'].value_counts()

In [12]:
# Choose a cutoff value and create a list of application types to be replaced\n",
# use the variable name `application_types_to_replace`\n",
color_types_to_replace = list(color_cnt[color_cnt < 10].index)
# Replace in dataframe
for color in color_types_to_replace:
    clean_df1['color'] = clean_df1['color'].replace(color,"Other")

In [13]:
for d in clean_df1.color.unique():
    # clean_df1['color'] = clean_df1['color'].apply(str)
    # compute Levensthein distance
    # and set to True if >= a 30
    # (you may have to play around with it)
    clean_df1[d] = clean_df1['color'].apply(
        lambda x : fuzz.ratio(x, d) >= 10
    )
    # set a name for the group
    # here, simply the shortest
    m = np.min(clean_df1[clean_df1[d]==True].color)
    # assign the group
    clean_df1.loc[clean_df1.color==d, 'color_group'] = m

In [14]:
def binning_function(col, cut_points, labels=None):
    minvalue = col.min()
    maxvalue = col.max()
    break_points = [minvalue] + cut_points + [maxvalue]
    print(break_points)
    if not labels:
        labels = range(len(cut_points) + 1)
    col_bin = pd.cut(col, bins=break_points, labels=labels, include_lowest=True)
    return col_bin

In [15]:
cut_points = [25000, 50000, 75000, 100000]
labels = ['< 25k', '25k - 50k', '50k - 75k', '75k - 100k', '> 100k']
clean_df1['vehicle_mileage_category'] = binning_function(clean_df1['mileage'], cut_points, labels)

[0.0, 25000, 50000, 75000, 100000, 999999.0]


In [16]:
clean_df1 = clean_df1[['auction_code', 'car_year', 'model_group', 'color_group', 'body', 'vehicle_mileage_category', 'condition_grade', 'times_run', 'sold_date_year', 'sold_date_month', 'sale_price', ]]
clean_df1.head()

,auction_code,car_year,model_group,color_group,body,vehicle_mileage_category,condition_grade,times_run,sold_date_year,sold_date_month,sale_price
1,AYCA,2009,TIGUAN 4X4,BEIGE,SUV,75k - 100k,34,1.0,2017,1,7100.0
2,AYCA,2009,ROUTAN,BEIGE,VAN,75k - 100k,31,1.0,2017,1,5800.0
3,AYCA,2014,JETTA,BLACK,4DSN,50k - 75k,27,1.0,2017,1,4500.0
6,NADE,2012,TIGUAN 4X4,BEIGE,SUV,75k - 100k,38,6.0,2017,1,8300.0
7,GOAA,2016,PASSAT,BEIGE,4DSN,< 25k,46,2.0,2017,1,13800.0


In [3]:
# df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15], ignore_index = True, sort = False)

In [17]:
# write out csv's to output folder
clean_df1.to_csv('../Output/Volkswagon.csv', index=False, header=True)
# clean_df2.to_csv('../Output/Toyota.csv', index=False, header=True)
# clean_df3.to_csv('../Output/Porsche.csv', index=False, header=True)
# clean_df4.to_csv('../Output/Mercedes.csv', index=False, header=True)
# clean_df5.to_csv('../Output/Maserati.csv', index=False, header=True)
# clean_df6.to_csv('../Output/Lincoln.csv', index=False, header=True)
# clean_df7.to_csv('../Output/Lexus.csv', index=False, header=True)